<a href="https://colab.research.google.com/github/andyarnell/Global_ecological_zones_mapping/blob/main/gez_comparison_sankey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Function to install a package if it's not already installed
def install_if_not_exists(package_name):
    try:
        __import__(package_name)
        print(f"{package_name} is already installed.")
    except ImportError:
        !pip install -q {package_name}
        print(f"{package_name} has been installed.")

install_if_not_exists("sankee")

import sankee
import ee
import ee # google earth engine

from google.colab import output
output.enable_custom_widget_manager()

gee_project_name = "ee-andyarnellgee"

ee.Authenticate()

ee.Initialize(project=gee_project_name)



In [ ]:
# Load two images
pre = ee.Image("GOOGLE/DYNAMICWORLD/V1/20200904T185921_20200904T190750_T10TEQ")
post = ee.Image("GOOGLE/DYNAMICWORLD/V1/20201009T190319_20201009T190349_T10TEQ")
aoi = ee.Geometry.Point([-122.30239918572622, 44.802882471354316]).buffer(1000)


In [ ]:
# Define the band name and the class labels and colors corresponding to each pixel value.
band = "label"

labels = {
    0: "water",
    1: "trees",
    2: "grass",
    3: "flooded_vegetation",
    4: "crops",
    5: "shrub_and_scrub",
    6: "built",
    7: "bare",
    8: "snow_and_ice"
}

palette = {
    0: "#419BDF",
    1: "#397D49",
    2: "#88B053",
    3: "#7A87C6",
    4: "#E49635",
    5: "#DFC35A",
    6: "#C4281B",
    7: "#A59B8F",
    8: "#B39FE1"
}


In [ ]:

# Generate the Sankey diagram from the two images
sankee.sankify(
    image_list=[pre, post],
    region=aoi,
    band=band,
    labels=labels,
    palette=palette,
)